In [1]:
import pandas as pd
import csv

In [3]:
df = pd.read_csv("spotify_millsongdata.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'spotify_millsongdata.csv'

In [5]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [6]:
df.isnull().sum()

,0
artist,0
song,0
link,0
text,0


In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df['text'][0]

"Where will I wander and wonder  \r\nNobody knows  \r\nBut wherever I'm a-going I'll go  \r\nIn search of a rose  \r\n  \r\nWhatever the will of the weather  \r\nAnd whether it shines or snows  \r\nWherever I'm a-going I'll go  \r\nIn search of a rose  \r\n  \r\nI don't know where it's found  \r\nI don't mind  \r\nAs long as the world spins around  \r\nI'll take my time  \r\n  \r\nI'll savour the softness of summer  \r\nI'll wrap up when winter blows  \r\nAnd wherever I'm going I'll go  \r\nIn search of a rose\r\n\r\n"

In [9]:
df.head(10)

,artist,song,text
0,Waterboys,In Search Of A Rose,Where will I wander and wonder \r\nNobody kno...
1,Yoko Ono,Loneliness,There're many things in life I can endure \r\...
2,Who,Glittering Girl,She wasn't a fool \r\nThat glittering girl \...
3,Smiths,How Soon Is Now,I am the son \r\nAnd the heir \r\nOf a shyne...
4,Wishbone Ash,Blowin' Free,I thought I had a girl \r\nI know because I s...
5,Stone Temple Pilots,Lady Picture Show,Lady picture show she hides behind the bedroom...
6,Tom T. Hall,Kentucky In The Morning,I will sing of a place that you may have seen ...
7,Xiu Xiu,Save Me Save Me,Though this is all you'll ever get \r\nIt wou...
8,Styx,Boat On The River,Take me back to my boat on the river \r\nI ne...
9,U2,The Fly,"Oh, baby child... \r\n \r\nIt's no secret th..."


In [11]:
#Text Cleaning/ Text Preprocessing

df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [14]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
nltk.download('punkt')

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [15]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [18]:
similarity[0]

array([1.        , 0.0014714 , 0.00970525, ..., 0.00259178, 0.09653195,
       0.09428475])

In [24]:
df[df['song'] == 'As Good As New']

def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)

    return songs

In [27]:
recommendation('As Good As New')

['Ma Baker',
 "Ma' Dear Ma' Dear",
 'The Last Laugh Of The Laughter',
 'A Bullet For Ramona',
 'My Hometown',
 'Light The Shade',
 'It Had To Be You',
 'Horribly Hazardous Heffalumps',
 'Lean Low',
 'Real Good Man',
 'You Give Good Love',
 'Love You Gently',
 'I Decided',
 'The Facts Of Life',
 "You're No Good",
 'Look At Me',
 'Forever',
 "Let's Roll",
 'New Way To Love',
 'By Myself']

In [22]:
def recommendation(song_df):
    # Check if the song exists in the DataFrame
    if df[df['song'] == song_df].empty:
        return "Song not found!"  # Or handle the case appropriately

    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)

    return songs

In [23]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))